In [2]:
!pip install opencv-python numpy svgwrite matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 806.6 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 1.4 MB/s eta 0:00:0000:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.4 MB/s eta 0:00:00a 0:00:01


In [15]:
import cv2
import numpy as np
from svgwrite import Drawing
import json

In [18]:
def create_comic_svg(image_path, target_color_hex, tolerance=30):
    # Convertir el color hex a BGR (OpenCV usa BGR)
    target_color_rgb = tuple(int(target_color_hex[i:i+2], 16) for i in (1, 3, 5))
    target_color_bgr = target_color_rgb[::-1]
    
    # Leer la imagen
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    
    # Crear máscara para el color específico
    lower_bound = np.array([max(0, c - tolerance) for c in target_color_bgr])
    upper_bound = np.array([min(255, c + tolerance) for c in target_color_bgr])
    mask = cv2.inRange(img, lower_bound, upper_bound)
    
    # Encontrar contornos
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Obtener dimensiones de la imagen
    height, width = img.shape[:2]
    
    # Crear SVG
    dwg = Drawing('comic_panels.svg', size=(width, height))
    
    # Crear diccionario para almacenar información de las viñetas
    panels_data = {
        "image_path": image_path,
        "panels": []
    }
    
    # Añadir cada contorno como un polígono en el SVG
    for i, contour in enumerate(contours):
        # Simplificar el contorno
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Convertir puntos a formato SVG
        points = []
        for point in approx:
            x = float(point[0][0])
            y = float(point[0][1])
            points.append((x, y))
        
        # Guardar información de la viñeta
        panel_info = {
            "id": i + 1,
            "coordinates": points,
            "description": ""  # Campo vacío para rellenar después
        }
        panels_data["panels"].append(panel_info)
        
        # Crear polígono SVG
        if points:  # Verificar que hay puntos antes de crear el polígono
            polygon = dwg.polygon(
                points=points,
                fill='none',
                stroke='red',
                stroke_width=1,
                class_='panel',
                onclick=f"handlePanelClick({i+1})"
            )
            dwg.add(polygon)
    
    # Guardar JSON
    with open('comic_panels.json', 'w', encoding='utf-8') as f:
        json.dump(panels_data, f, ensure_ascii=False, indent=4)
    
    # Guardar SVG
    dwg.save()
    
    return len(contours)

In [19]:
# Ejemplo de uso
image_path = "test.jpg"  # Reemplaza esto con tu ruta real
target_color = "#82c173"  # Color de los bordes a detectar (rojo en este ejemplo)

num_panels = create_comic_svg(image_path, target_color)
print(f"Se detectaron {num_panels} viñetas")

Se detectaron 2 viñetas


In [13]:
# Para visualizar la imagen original y la máscara
import matplotlib.pyplot as plt

def visualizar_resultados(image_path, mask):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.imshow(img_rgb)
    ax1.set_title('Imagen Original')
    ax1.axis('off')
    
    ax2.imshow(mask, cmap='gray')
    ax2.set_title('Máscara de Detección')
    ax2.axis('off')
    
    plt.show()